# Try the Student Orientation Model

This notebook loads the trained model and encoders from the `model/` directory and performs predictions on new student data.

In [3]:
import joblib
import pandas as pd
import numpy as np
import os

# 1. Load the Model and Encoders
model_path = 'model/best_model_Decision_Tree.pkl'
le_filiere_path = 'model/label_encoder_filiere.pkl'
le_rec_path = 'model/label_encoder_recommendation.pkl'

if not os.path.exists(model_path):
    print("\u26a0\ufe0f Model files not found. Make sure you have run the training notebook first.")
else:
    print("\u231b Loading model artifacts...")
    model = joblib.load(model_path)
    le_filiere = joblib.load(le_filiere_path)
    le_rec = joblib.load(le_rec_path)
    print("\u2705 Model and Encoders loaded successfully!")

⌛ Loading model artifacts...
✅ Model and Encoders loaded successfully!


In [2]:
def predict_orientation(moyenne_gen, maths, physique, francais, filiere):
    """
    Predicts the recommended orientation based on student grades.
    """
    # 1. Prepare Data
    input_data = pd.DataFrame({
        'Moyenne_Generale': [moyenne_gen],
        'Note_Maths': [maths],
        'Note_Physique': [physique],
        'Note_Francais': [francais],
        'Filiere_Bac': [filiere]
    })
    
    try:
        # 2. Encode Filiere (using the loaded encoder)
        input_data['Filiere_Bac_Encoded'] = le_filiere.transform(input_data['Filiere_Bac'])
        
        # 3. Select Features (Ensure order matches training)
        features = ['Moyenne_Generale', 'Note_Maths', 'Note_Physique', 'Note_Francais', 'Filiere_Bac_Encoded']
        X = input_data[features]
        
        # 4. Predict
        prediction_encoded = model.predict(X)
        
        # 5. Decode Prediction
        recommendation = le_rec.inverse_transform(prediction_encoded)[0]
        
        return recommendation, input_data
    except Exception as e:
        return f"Error: {str(e)}", None

In [3]:
# Test Case 1: Excellent Profile (Science)
rec, _ = predict_orientation(17.5, 18, 17, 16, 'SM-B')
print(f"Student 1 (17.5 Avg, SM-B) -> Recommendation: {rec}")

# Test Case 2: Average Profile (Physics)
rec, _ = predict_orientation(13.5, 14, 13, 12, 'PC')
print(f"Student 2 (13.5 Avg, PC)   -> Recommendation: {rec}")

# Test Case 3: Economics Profile
rec, _ = predict_orientation(14.0, 10, 10, 16, 'Eco-Gestion')
print(f"Student 3 (14.0 Avg, Eco)  -> Recommendation: {rec}")

Student 1 (17.5 Avg, SM-B) -> Recommendation: Medecine / Pharmacie
Student 2 (13.5 Avg, PC)   -> Recommendation: FST
Student 3 (14.0 Avg, Eco)  -> Recommendation: ENCG


In [4]:
# Interactive Loop (Optional)
print("\n--- Interactive Prediction ---")
try:
    # Example values for a quick test run without user input prompts blocking execution
    test_input = {'m': 15, 'math': 15, 'phy': 15, 'fr': 14, 'fil': 'PC'}
    
    res, _ = predict_orientation(test_input['m'], test_input['math'], test_input['phy'], test_input['fr'], test_input['fil'])
    print(f"Test Input {test_input} -> {res}")
except:
    pass


--- Interactive Prediction ---
Test Input {'m': 15, 'math': 15, 'phy': 15, 'fr': 14, 'fil': 'PC'} -> ENSA / ENSAM


In [4]:
def predict_orientation(moyenne_gen, maths, physique, francais, filiere):
    """
    Predicts the recommended orientation AND returns probabilities (%) for each class.
    """

    # 1. Prepare Data
    input_data = pd.DataFrame({
        'Moyenne_Generale': [moyenne_gen],
        'Note_Maths': [maths],
        'Note_Physique': [physique],
        'Note_Francais': [francais],
        'Filiere_Bac': [filiere]
    })

    try:
        # 2. Encode Filiere
        input_data['Filiere_Bac_Encoded'] = le_filiere.transform(input_data['Filiere_Bac'])

        # 3. Select Features (same order as training)
        features = [
            'Moyenne_Generale',
            'Note_Maths',
            'Note_Physique',
            'Note_Francais',
            'Filiere_Bac_Encoded'
        ]
        X = input_data[features]

        # 4. Predict class
        prediction_encoded = model.predict(X)
        recommendation = le_rec.inverse_transform(prediction_encoded)[0]

        # 5. Predict probabilities
        probabilities = model.predict_proba(X)[0]   # shape: (n_classes,)

        # 6. Convert probabilities to percentages with labels
        class_labels = le_rec.inverse_transform(model.classes_)
        proba_percent = {
            class_labels[i]: round(probabilities[i] * 100, 2)
            for i in range(len(class_labels))
        }

        return recommendation, proba_percent

    except Exception as e:
        return f"Error: {str(e)}", None


In [5]:
rec, probs = predict_orientation(17.5, 18, 17, 16, 'SM-B')
print("Recommendation:", rec)
print("Probabilities:", probs)


Recommendation: Medecine / Pharmacie
Probabilities: {'CPGE (Prépas)': 0.0, 'ENCG': 0.0, 'ENSA / ENSAM': 0.0, 'FST': 0.0, 'Faculté (Licence Fondamentale)': 0.0, 'Medecine / Pharmacie': 100.0}
